In [10]:
from pathlib import Path
from collections import Counter
from PIL import Image, ImageOps
from tqdm import tqdm
import sys

In [11]:
def scan_resolutions(src_dir: Path):
    """src_dir 하위 PNG 파일 해상도(폭,높이) 집계 → Counter 반환"""
    sizes = Counter()
    for png in src_dir.rglob("*.png"):
        try:
            with Image.open(png) as im:
                sizes[im.size] += 1       # (width, height)
        except Exception as e:
            print(f"[WARN] {png}: {e}", file=sys.stderr)
    return sizes

In [12]:
def resize_dataset(src_dir: Path, dst_dir: Path, target: int, keep_ratio=True):
    """
    src_dir → dst_dir 동일 구조 유지하며 정사각형 target×target 저장
      • keep_ratio=True  : 비율 유지 + 검은 여백 채움(Pad)
      • keep_ratio=False : 강제 Stretch
    """
    dst_dir.mkdir(parents=True, exist_ok=True)

    for png in tqdm(list(src_dir.rglob("*.png")), desc="Resizing"):
        rel_path = png.relative_to(src_dir)
        save_path = dst_dir / rel_path
        save_path.parent.mkdir(parents=True, exist_ok=True)

        with Image.open(png) as im:
            if keep_ratio:
                im = ImageOps.pad(
                    im, (target, target),
                    color=(0, 0, 0),        # 여백 색상: 검정
                    centering=(0.5, 0.5),
                    method=Image.Resampling.LANCZOS
                )
            else:
                im = im.resize((target, target), Image.Resampling.LANCZOS)
            im.save(save_path)

In [13]:
SRC_DIR = Path('blast/split')
DST_DIR = Path('blast/split_vit224')
TARGET = 224

In [14]:
res_counter = scan_resolutions(SRC_DIR)

print("\n[해상도 분포]")
for (w, h), cnt in sorted(res_counter.items(), key=lambda x: -x[1]):
    print(f"  {w:4d}×{h:4d} : {cnt}장")

# --- 리사이즈 필요 여부 판단 ---
all_same_target = (
    len(res_counter) == 1 and
    next(iter(res_counter)).count(TARGET) == 2
)

if all_same_target:
    print("\n모든 이미지가 이미 목표 해상도입니다. 리사이즈를 건너뜁니다.")
else:
    # 필요 시 yes/no 입력받기
    proceed = input(f"\n{TARGET}×{TARGET} 로 리사이즈 후 {DST_DIR} 에 저장할까요? [y/N] ")
    if proceed.lower() == "y":
        print(f"\n[2] 리사이즈 시작…")
        resize_dataset(SRC_DIR, DST_DIR, TARGET, keep_ratio=True)
        print("✅ 완료!")
    else:
        print("⏩ 리사이즈를 취소했습니다.")


[해상도 분포]
   512× 384 : 641장

[2] 리사이즈 시작…


Resizing: 100%|██████████| 641/641 [00:11<00:00, 54.01it/s]

✅ 완료!
